<h3>Import Required Libraries:

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


<h3>Data Scraping:

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai").text

In [3]:
soup = BeautifulSoup(data, 'html.parser')

In [4]:
neighborhoodList = []

In [5]:
for row in soup.find_all("div", class_="toc")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
import re 
  
def remove(mum): 
    pattern = '[0-9.]'
    mum = [re.sub(pattern, '', i) for i in mum] 
    return mum

neighborhoodList = remove(neighborhoodList)

In [7]:
mum = pd.DataFrame({"Neighborhood": neighborhoodList})
mum

,Neighborhood
0,Western Suburbs\n\n Andheri\n Bhayandar\n Ban...
1,Andheri
2,Bhayandar
3,Bandra
4,Borivali
5,Dahisar
6,Goregaon
7,Jogeshwari
8,Juhu
9,Kandivali west


In [8]:
mum = mum.drop([0, 26, 31,43, 44,17])
mum

,Neighborhood
1,Andheri
2,Bhayandar
3,Bandra
4,Borivali
5,Dahisar
6,Goregaon
7,Jogeshwari
8,Juhu
9,Kandivali west
10,Kandivali east


In [9]:
mum = mum.reset_index(drop = True)

In [10]:
mum

,Neighborhood
0,Andheri
1,Bhayandar
2,Bandra
3,Borivali
4,Dahisar
5,Goregaon
6,Jogeshwari
7,Juhu
8,Kandivali west
9,Kandivali east


In [11]:
mum.shape

(39, 1)

<h3>Getting Coordinates of each Location:

In [12]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [13]:
coords = [ get_latlng(neighborhood) for neighborhood in mum["Neighborhood"].tolist() ]
coords

[[19.118459378296492, 72.84176321065843],
 [19.307430000000068, 72.85184000000004],
 [19.054370000000063, 72.84017000000006],
 [19.229360000000042, 72.85751000000005],
 [19.250030000000038, 72.85907000000003],
 [19.164550000000077, 72.84946000000008],
 [19.137920000000065, 72.84941000000003],
 [19.014920000000075, 72.84522000000004],
 [19.207100000000025, 72.83498000000003],
 [19.205760000000055, 72.86953000000005],
 [19.06913000000003, 72.84640000000007],
 [19.186550000000068, 72.84842000000003],
 [19.081770000000063, 72.84205000000003],
 [19.07934000000006, 72.83916000000005],
 [19.01657000000006, 72.85853000000003],
 [19.100630000000024, 72.84378000000004],
 [19.145560000000046, 72.94856000000004],
 [19.08652321008152, 72.90900774216628],
 [19.131380000000036, 72.93568000000005],
 [19.064980000000048, 72.88069000000007],
 [19.171830000000057, 72.95565000000005],
 [19.123100000000022, 72.90942000000007],
 [19.023270726614363, 72.84390704954123],
 [19.111090000000047, 72.9278100000000

In [14]:
df_new = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [15]:
mum['Latitude'] = df_new['Latitude']
mum['Longitude'] = df_new['Longitude']

In [16]:
mum      #Final dataframe along with coords

,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Bhayandar,19.307430,72.851840
2,Bandra,19.054370,72.840170
3,Borivali,19.229360,72.857510
4,Dahisar,19.250030,72.859070
5,Goregaon,19.164550,72.849460
6,Jogeshwari,19.137920,72.849410
7,Juhu,19.014920,72.845220
8,Kandivali west,19.207100,72.834980
9,Kandivali east,19.205760,72.869530


In [17]:
 address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Latitude: {}, Longitude: {}.'.format(latitude, longitude))

Latitude: 18.9387711, Longitude: 72.8353355.


<h3>Map of each neighborhood:

In [18]:
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(mum['Latitude'], mum['Longitude'], mum['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mumbai)  
    
map_mumbai

In [48]:
# define Foursquare Credentials and Version
CLIENT_ID = '********;)' # your Foursquare ID
CLIENT_SECRET = '********;)' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ********;)
CLIENT_SECRET:********;)


<h3>Foursquare API used with radius = 2000:

In [20]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mum['Latitude'], mum['Longitude'], mum['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [21]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3142, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118459,72.841763,Naturals,19.111204,72.837255,Ice Cream Shop
3,Andheri,19.118459,72.841763,Narayan Sandwich,19.121398,72.850270,Sandwich Place
4,Andheri,19.118459,72.841763,Temptations,19.113767,72.841337,Ice Cream Shop


In [23]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 209 uniques categories.


In [25]:
# print out the list of categories
venues_df['VenueCategory'].unique()

array(['Bakery', 'Indian Restaurant', 'Ice Cream Shop', 'Sandwich Place',
       'Pub', 'Falafel Restaurant', 'Coffee Shop', 'Juice Bar',
       'Pizza Place', 'Seafood Restaurant', 'Multiplex', 'Snack Place',
       'Fast Food Restaurant', 'Chinese Restaurant', 'Breakfast Spot',
       'Maharashtrian Restaurant', 'Café', 'American Restaurant',
       'Cocktail Bar', 'Diner', 'Bar', 'Electronics Store', 'Spa',
       'Gym / Fitness Center', 'BBQ Joint', 'Asian Restaurant', 'Lounge',
       "Women's Store", 'Residential Building (Apartment / Condo)',
       'Smoke Shop', 'Food Truck', 'Vegetarian / Vegan Restaurant',
       'Park', 'Liquor Store', 'Athletics & Sports', 'Fish Market',
       'Martial Arts Dojo', 'Tea Room', 'Train Station', 'Restaurant',
       'Playground', 'Gym', 'Dessert Shop', 'Sports Club', 'Gourmet Shop',
       'Korean Restaurant', 'Deli / Bodega', 'Salad Place',
       'German Restaurant', 'French Restaurant', 'College Auditorium',
       'Modern European Restaur

<h3>Analysis of each neighborhood:

In [28]:
onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

print(onehot.shape)
onehot.head()

(3142, 210)


,Neighborhoods,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [30]:
grouped = onehot.groupby(["Neighborhoods"]).mean().reset_index()
print(grouped.shape)
grouped

(39, 210)


,Neighborhoods,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Andheri,0.000000,0.00,0.010000,0.00,0.000000,0.00,0.000000,0.020000,0.010000,0.000000,0.010000,0.00,0.020000,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.060000,0.000000,0.00,0.02,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.020000,0.000000,0.010000,0.000000,0.000000,0.010000,0.00,0.00000,0.020000,0.00,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.050000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.170000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.010000,0.020000,0.010000,0.000000,0.01,0.000000

In [31]:
len(grouped[grouped["Café"] > 0])

36

In [33]:
cafe = grouped[["Neighborhoods","Café"]]    #dataframe for cafe
cafe.head()

,Neighborhoods,Café
0,Andheri,0.040000
1,Antop Hill,0.064103
2,Bandra,0.100000
3,Bhandup,0.040000
4,Bhayandar,0.000000


<h3>Clustering:

In [34]:
# set number of clusters
kclusters = 3

clustering = cafe.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 0, 2, 1, 0, 0, 1, 0])

In [35]:
merged = cafe.copy()

# add clustering labels
merged["Cluster Labels"] = kmeans.labels_

In [36]:
merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
merged.head()

,Neighborhood,Café,Cluster Labels
0,Andheri,0.040000,0
1,Antop Hill,0.064103,1
2,Bandra,0.100000,1
3,Bhandup,0.040000,0
4,Bhayandar,0.000000,2


In [39]:
merged = merged.join(mum.set_index("Neighborhood"), on="Neighborhood")

print(merged.shape)
merged.head() 

(39, 5)


,Neighborhood,Café,Cluster Labels,Latitude,Longitude
0,Andheri,0.040000,0,19.118459,72.841763
1,Antop Hill,0.064103,1,19.026140,72.866450
2,Bandra,0.100000,1,19.054370,72.840170
3,Bhandup,0.040000,0,19.145560,72.948560
4,Bhayandar,0.000000,2,19.307430,72.851840


In [40]:
print(merged.shape)
merged.sort_values(["Cluster Labels"], inplace=True)
merged

(39, 5)


,Neighborhood,Café,Cluster Labels,Latitude,Longitude
0,Andheri,0.040000,0,19.118459,72.841763
34,Vasai,0.050000,0,19.079340,72.839160
30,Powai,0.060241,0,19.123100,72.909420
29,Parel,0.050000,0,18.995700,72.839130
25,Malad,0.031579,0,19.186550,72.848420
24,Kurla,0.033708,0,19.064980,72.880690
23,Khar,0.040000,0,19.069130,72.846400
21,Kandivali west,0.040000,0,19.207100,72.834980
20,Kandivali east,0.044118,0,19.205760,72.869530
37,Vile Parle,0.040000,0,19.100630,72.843780


In [41]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h4>Cluster 1:

In [43]:
merged.loc[merged['Cluster Labels'] == 0]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
0,Andheri,0.040000,0,19.118459,72.841763
34,Vasai,0.050000,0,19.079340,72.839160
30,Powai,0.060241,0,19.123100,72.909420
29,Parel,0.050000,0,18.995700,72.839130
25,Malad,0.031579,0,19.186550,72.848420
24,Kurla,0.033708,0,19.064980,72.880690
23,Khar,0.040000,0,19.069130,72.846400
21,Kandivali west,0.040000,0,19.207100,72.834980
20,Kandivali east,0.044118,0,19.205760,72.869530
37,Vile Parle,0.040000,0,19.100630,72.843780


<h4>Cluster 2:

In [44]:
merged.loc[merged['Cluster Labels'] == 1]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
38,Virar,0.076087,1,19.016570,72.858530
2,Bandra,0.100000,1,19.054370,72.840170
1,Antop Hill,0.064103,1,19.026140,72.866450
27,Matunga,0.071429,1,19.027200,72.855890
26,Mankhurd,0.071429,1,19.048530,72.932220
22,Kanjurmarg,0.071429,1,19.131380,72.935680
31,Santacruz,0.093750,1,19.081770,72.842050
8,Colaba,0.080000,1,18.915420,72.825920
35,Vidyavihar,0.070000,1,19.023271,72.843907
15,Govandi,0.089286,1,19.056170,72.914830


<h4>Cluster 3:

In [46]:
merged.loc[merged['Cluster Labels'] == 2]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
33,Trombay,0.000000,2,19.01900,72.89799
13,Girgaon,0.020000,2,18.95696,72.81945
28,Mulund,0.025974,2,19.17183,72.95565
16,Jogeshwari,0.000000,2,19.13792,72.84941
14,Goregaon,0.020000,2,19.16455,72.84946
32,Tardeo,0.010000,2,18.97254,72.81478
4,Bhayandar,0.000000,2,19.30743,72.85184


<h4>Analysis:</h4><br>
<p>     Cluster 3 consists of all the neighborhoods with less number of cafes. Most of the cafes are present in neighborhood of cluster 2. Cluster 1 has moderate number of cafes. Hence, according to me, new cafes should be opened in cluster 3 ares.